THIS IS CODE FOR SOLVING TRACK 1 (CLASSIFICATION OF SPORT VIDEOS)




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Discard the output of this cell.
%%capture
# Install the required libraries.
!pip install keras

In [ ]:
# Import the required libraries.
import os
import cs
import cv2
import math
import keras
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
from keras.layers import TimeDistributed,BatchNormalization, SeparableConv2D,Input,Conv2D, MaxPooling2D, Dropout, Flatten, LSTM, Dense,Conv3D, MaxPooling3D, GlobalAveragePooling2D
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow. keras.utils import to_categorical
from tensorflow. keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import plot_model

In [ ]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT, IMAGE_WIDTH = 224,224
# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 32
# Specify the directory containing the UCF50 dataset.
DATASET_DIR1 = "/content/drive/MyDrive/Colab Notebooks/Few Shot video classification/Train"
DATASET_DIR2 = "/content/drive/MyDrive/Colab Notebooks/Few Shot video classification/Test/Support-set"
# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST= ["football", "judo", "rugby", "squash","wrestling"]

In [ ]:
def frames_extraction (video_path):
  '''
  This function will extract the required frames from a video after resizing and normalizing them.
  Args:
  video_path: The path of the video in the disk, whose frames are to be extracted.
  Returns:
  frames list: A list containing the resized and normalized frames of the video.
  111
  '''
#Declare a list to store video frames.
  frames_list = []
#Read the Video File using the VideoCapture object.
  video_reader = cv2.VideoCapture(video_path)
# Get the total number of frames in the video.
  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
  print(video_frames_count)
# Calculate the the interval after which frames will be added to the list..
  skip_frames_window = max(int (video_frames_count/SEQUENCE_LENGTH), 1)
# Iterate through the Video Frames.
  for frame_counter in range(SEQUENCE_LENGTH):
# Set the current frame position of the video.
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter *skip_frames_window)
# Reading the frame from the video.
    success, frame =video_reader.read()
# Check if Video frame is not successfully read then break the loop
    if not success:
      break
# Resize the Frame to fixed height and width.
    resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
# Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
    normalized_frame = resized_frame / 255
#Append the normalized frame into the frames list.
    frames_list.append(normalized_frame)
# Release the VideoCapture object.
  video_reader.release()
#Return the frames list.
  return frames_list

In [ ]:
def create_dataset():
  '''
  This function will extract the data of the selected classes and create the required dataset.
  Returns:
  features: A list containing the extracted frames of the videos.
  labels: A list containing the indexes of the classes associated with the videos.
  video_files_paths: A list containing the paths of the videos in the disk.
  '''
# Declared Empty Lists to store the features, labels and video file path values.
  features = []
  labels = []
  video_files_paths = []
# Iterating through all the classes mentioned in the classes list
  for class_index, class_name in enumerate (CLASSES_LIST):
# Display the name of the class whose data is being extracted.
    print (f'Extracting Data of Class: {class_name}')
# Get the list of video files present in the specific class name directory.
    files_list= os.listdir (os.path.join(DATASET_DIR1, class_name))
# Iterate through all the files present in the files list.
    for file_name in files_list:
# Get the complete video path.
      video_file_path = os.path.join(DATASET_DIR1, class_name, file_name)
#Extract the frames of the video file.
      frames=frames_extraction (video_file_path)
# Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
# So ignore the vides having frames less than the SEQUENCE_LENGTH.
      print(len(frames))
      if len(frames) == SEQUENCE_LENGTH:
# Append the data to their repective lists.
        print("\n")
        features.append(frames)
        labels.append(class_index)
        video_files_paths.append(video_file_path)
      if len(frames) < SEQUENCE_LENGTH:
        # Get the last frame
        last_frame = frames[-1] if frames else None
        # Duplicate the last frame until the sequence reaches the desired length
        while len(frames) < SEQUENCE_LENGTH and last_frame is not None:
            frames.append(last_frame)
        print("\n")
        features.append(frames)
        labels.append(class_index)
        video_files_paths.append(video_file_path)
        print(len(frames))
# Converting the list to numpy arrays

  features=np.asarray(features)
  labels=np.array (labels)
#Return the frames, class index, and video file path.
  return features, labels, video_files_paths

In [ ]:
def create_dataset_test():
  '''
  This function will extract the data of the selected classes and create the required dataset.
  Returns:
  features: A list containing the extracted frames of the videos.
  labels: A list containing the indexes of the classes associated with the videos.
  video_files_paths: A list containing the paths of the videos in the disk.
  '''
# Declared Empty Lists to store the features, labels and video file path values.
  features = []
  labels = []
  video_files_paths = []
  c=0
# Iterating through all the classes mentioned in the classes list
  for class_index in range(1,len(CLASSES_LIST)):
# Display the name of the class whose data is being extracted.
    print (f'Extracting Data of Class: rugby')
# Get the list of video files present in the specific class name directory.
#/content/drive/MyDrive/Colab Notebooks/few-shot-video-classification/Few Shot video classification/Test/Support-set/0
    files_list= os.listdir (os.path.join(DATASET_DIR2, str(class_index-1)))
# Iterate through all the files present in the files list.
    for file_name in files_list:
# Get the complete video path.
      video_file_path = os.path.join(DATASET_DIR2,str(class_index-1), file_name)
#Extract the frames of the video file.
      frames=frames_extraction (video_file_path)
# Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
# So ignore the vides having frames less than the SEQUENCE_LENGTH.
      print(len(frames))
      if len(frames) == SEQUENCE_LENGTH:
# Append the data to their repective lists.
        print("\n")

        features.append(frames)
        labels.append(c)
        video_files_paths.append(video_file_path)
      if len(frames) < SEQUENCE_LENGTH:
        # Get the last frame
        last_frame = frames[-1] if frames else None
        # Duplicate the last frame until the sequence reaches the desired length
        while len(frames) < SEQUENCE_LENGTH and last_frame is not None:
            frames.append(last_frame)
        print("\n")
        features.append(frames)
        labels.append(c)
        video_files_paths.append(video_file_path)
        print(len(frames))
# Converting the list to numpy arrays
    c=c+1
  features=np.asarray(features)
  labels=np.array (labels)
#Return the frames, class index, and video file path.
  return features, labels, video_files_paths

In [ ]:
# Create the dataset.
featuress, labelss, video_files_pathss=create_dataset_test()
features_test=featuress
labels_test=labelss

In [ ]:
# Create the dataset.
features, labels, video_files_paths=create_dataset()
features_train=features
labels_train=labels

In [ ]:
LEARNING_RATE = 0.0005

In [ ]:
from sklearn.model_selection import train_test_split
# Define the proportion of data to be used for validation
validation_split = 0.20  # 20% for validation
# Split the data
train_data, val_data, train_labels, val_labels = train_test_split(
    features_train,
    labels_train,
    test_size=validation_split,
    random_state=42  # Ensures reproducibility
)
print(train_labels.shape)
print(val_labels.shape)

In [ ]:
#chaecking and adjusting the labels shape
print(train_labels.shape)
print(val_labels.shape)
print(labels_test.shape)
train_labels =to_categorical(train_labels, 5)
val_labels =to_categorical(val_labels, 5)
labels_test =to_categorical(labels_test, 4)
print(train_labels.shape)
print(val_labels.shape)
print(labels_test.shape)

In [ ]:
# Define a function to load all videos from a query folder
def load_query_videos_from_folder(query_folder_path, img_height, img_width, sequence_length):
    query_videos = []
    data_gen = ImageDataGenerator(rescale=1./255)

    for video_file in os.listdir(query_folder_path):
        video_path = os.path.join(query_folder_path, video_file)
        video_reader = cv2.VideoCapture(video_path)

        frames = []
        video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
        skip_frames_window = max(int(video_frames_count / sequence_length), 1)

        for frame_counter in range(sequence_length):
            video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
            success, frame = video_reader.read()
            if not success:
                break
            resized_frame = cv2.resize(frame, (img_height, img_width))
            normalized_frame = resized_frame / 255.0
            frames.append(normalized_frame)

        while len(frames) < sequence_length:
            frames.append(frames[-1])

        query_videos.append(np.array(frames))
        video_reader.release()

    return np.array(query_videos)

# Define the path query dataset folder
query_folder_path = "/content/drive/MyDrive/Colab Notebooks/Few Shot video classification/Test/Query"

# Load query videos from the folder
query_data = load_query_videos_from_folder(query_folder_path, IMAGE_HEIGHT, IMAGE_WIDTH, SEQUENCE_LENGTH)

In [ ]:
files_list1= os.listdir (os.path.join("/content/drive/MyDrive/Colab Notebooks/Few Shot video classification/Test", "Query"))
# Iterate through all the files present in the files list.
query_video_paths = []
for file_name in files_list1:
# Get the complete video path.
  video_file_path1 = os.path.join("/content/drive/MyDrive/Colab Notebooks/Few Shot video classification/Test/Query", file_name)
  i=video_file_path1.rfind('_')
  j=video_file_path1.rfind('.')
  query_video_paths.append(int(video_file_path1[i+1:j]))
#Extract the frames of the video file.

UPTO THIS I HAVE INITIALISED ALL FRAMES AND LABEL OF THE TRAIN,SUPPORT AND QUERY FOLDER AND ADJUST THEIR SHAPES AS REQUIRED


In [ ]:
def create_lrcn_with_resnet(input_shape=(32, 224, 224, 3), num_classes=5):
    model = Sequential()

    resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in resnet_base.layers[:-30]:
        layer.trainable = False  # Freeze early layers initially

    model.add(TimeDistributed(resnet_base, input_shape=input_shape))
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.3)))

    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))))
    model.add(TimeDistributed(BatchNormalization()))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.3)))

    model.add(TimeDistributed(GlobalAveragePooling2D()))

    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.3))

    model.add(Dense(4, activation='softmax'))

    return model

In [ ]:
class MAML:
    def __init__(self, model, learning_rate=1e-4, meta_lr=1e-3, num_adapt_steps=5):
        self.model = model
        self.learning_rate = learning_rate
        self.meta_lr = meta_lr
        self.num_adapt_steps = num_adapt_steps
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.meta_lr)

    def loss_function(self, y_true, y_pred):
        """Calculate categorical cross-entropy loss."""
        return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred))

    def adapt(self, x_support, y_support):
        """Perform inner-loop adaptation using the support set."""
        for step in range(self.num_adapt_steps):
            initial_weights = [tf.identity(var) for var in self.model.trainable_variables]
            print("Support Shape:", x_support.shape)
            print("Support Labels Shape:", y_support.shape)

            with tf.GradientTape() as tape:
                predictions = self.model(x_support, training=True)
                loss = self.loss_function(y_support, predictions)

            gradients = tape.gradient(loss, self.model.trainable_variables)

            # Log the gradient norms for debugging
            gradient_norms = [tf.norm(grad).numpy() for grad in gradients if grad is not None]
            print(f"Adaptation Step {step + 1}, Gradient Norms: {gradient_norms}")

            # Update model weights
            for var, grad in zip(self.model.trainable_variables, gradients):
                var.assign_sub(self.learning_rate * grad)

            weight_diffs = [tf.norm(var - init_var).numpy() for var, init_var in zip(self.model.trainable_variables, initial_weights)]
            print(f"Adaptation Step {step + 1}, Weight Differences: {weight_diffs}")

    def meta_train_step(self, task_batch):
        """Perform one meta-training step over a batch of tasks."""
        meta_gradients = [tf.zeros_like(weight) for weight in self.model.trainable_variables]

        for task_data in task_batch:
            x_support, y_support, x_query, y_query = task_data
            print(y_support)
            print("Support Shape:", x_support.shape)
            print("Support Labels Shape:", y_support.shape)
            print("Query Shape:", x_query.shape)
            print("Query Labels Shape:", y_query.shape)
            # Adapt to support set

            self.adapt(x_support, y_support)

            with tf.GradientTape() as tape:
                predictions = self.model(x_query, training=True)
                loss = self.loss_function(y_query, predictions)

            task_gradients = tape.gradient(loss, self.model.trainable_variables)
            meta_gradients = [meta_g + task_g for meta_g, task_g in zip(meta_gradients, task_gradients)]

        self.optimizer.apply_gradients(zip(meta_gradients, self.model.trainable_variables))

    def train(self, task_batch, epochs, validation_data=None, patience=5):
      """Train the model using meta-learning approach."""
      best_val_loss = float('inf')
      wait = 0

      for epoch in range(epochs):
        for batch in task_batch:
            self.meta_train_step(batch)
        print(f"Epoch {epoch + 1}/{epochs} complete.")

        # Evaluate on validation data
        if validation_data:
            val_support_data, val_support_labels, val_query_data, val_query_labels = validation_data
            val_predictions = self.predict_query(val_support_data, val_support_labels, val_query_data)

            # Compute validation loss
            val_loss = self.loss_function(val_query_labels, val_predictions)
            print(f"Validation Loss: {val_loss.numpy()}")

            # Compute validation accuracy
            val_true_labels = np.argmax(val_query_labels, axis=1)
            val_accuracy = np.mean(val_predictions == val_true_labels)
            print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                wait = 0
            else:
                wait += 1
                if wait >= patience:
                    print("Early stopping triggered.")
                    return

    def predict_query(self, x_support, y_support, x_query):
        """Adapt to the support set and predict the labels for the query set."""
        self.adapt(x_support, y_support)
        print("x_query shape:", x_query.shape)
        print("Model input shape:", self.model.input_shape)

        predictions = self.model.predict(x_query, verbose=1)
        predicted_labels = np.argmax(predictions, axis=1)

        return predicted_labels



In [ ]:
def predict_on_query_set(maml, support_data, support_labels, query_data, query_video_paths, output_csv_file):
    """Adapt model on support set and predict query data, saving results to CSV."""
    print("Adapting model to support set...")
    predicted_classes = maml.predict_query(support_data, support_labels, query_data)

    try:
        print(f"Saving predictions to {output_csv_file}...")
        with open(output_csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['VID_ID', 'LABEL'])

            for video_path, predicted_class in zip(query_video_paths, predicted_classes):
                writer.writerow([video_path, predicted_class])

        print("Predictions saved successfully.")
    except Exception as e:
        print(f"Error saving predictions: {e}")

In [ ]:
def create_task_batch(train_data, train_labels, query_data, query_labels, batch_size=2, sequence_length=40):
    task_batch = []
    num_tasks = len(train_data) // batch_size

    for i in range(num_tasks):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        x_support = train_data[start_idx:end_idx]
        y_support = train_labels[start_idx:end_idx]

        query_start_idx = (i) * batch_size
        query_end_idx = min((i + 1) * batch_size, len(query_data))

        if query_start_idx >= len(query_data):
            break

        x_query = query_data[query_start_idx:query_end_idx]
        y_query = query_labels[query_start_idx:query_end_idx]

        print(y_support)
        print(y_query)
        # Add a new axis to include the batch dimension
        print("Support Shape:", x_support.shape)
        print("Support Labels Shape:", y_support.shape)
        print("Query Shape:", x_query.shape)
        print("Query Labels Shape:", y_query.shape)
        dem=[]
        dem.append(x_support)
        dem.append(y_support)
        dem.append(x_query)
        dem.append(y_query)

        task_batch.append([dem])

    return task_batch


In [ ]:
# Loading your actual data here
support_data = features_test  # Loading support data
support_labels = labels_test    # Loading support labels

# Create task batch
task_batch = create_task_batch(support_data,support_labels,support_data,support_labels)
print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")# for better understanding
#checking the task batch
for batch in task_batch:
   for task_data in batch:
            x_support, y_support, x_query, y_query = task_data
            print("Support Shape:", x_support.shape)
            print("Support Labels Shape:", y_support.shape)
            print("Query Shape:", x_query.shape)
            print("Query Labels Shape:", y_query.shape)
#intilialising maml
maml = MAML(create_lrcn_with_resnet(), learning_rate=1e-4, meta_lr=1e-3, num_adapt_steps=3)
print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")# for better understanding
#training
maml.train(task_batch, epochs=5)
print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")# for better understandin

output_csv_file = 'predictionsfinal.csv'#name of the folder to save
# Predict and save to CSV
predict_on_query_set(maml, support_data, support_labels, query_data, query_video_paths, output_csv_file)

THIS IS THE SECOND WAY OF APPROACHING THE QUESTION

In [ ]:
def create_LRCN_model():
    model = Sequential()

    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same', activation='relu'), input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(128, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model


In [ ]:
# Construct the required LRCN model.
LRCN_model=create_LRCN_model()
# Display the success message.
print("Model Created Successfully!")

In [ ]:
# Create an Instance of Early Stopping Callback.
# Learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return 1e-4
    elif epoch < 20:
        return 1e-5
    else:
        return 1e-6
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min',restore_best_weights=True)
# Set a higher learning rate
LRCN_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Start training the model..
LRCN_model_training_history = LRCN_model.fit(train_data, train_labels, epochs=70, validation_data=(val_data, val_labels), batch_size =4, callbacks=[early_stopping, LearningRateScheduler(lr_scheduler)])

In [ ]:
class MAML:
    def __init__(self, model, learning_rate=1e-4, meta_lr=1e-3, num_adapt_steps=5):
        self.model = model
        self.learning_rate = learning_rate
        self.meta_lr = meta_lr
        self.num_adapt_steps = num_adapt_steps
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.meta_lr)

    def loss_function(self, y_true, y_pred):
        """Calculate categorical cross-entropy loss."""
        return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred))

    def adapt(self, x_support, y_support, num_support_classes):
        """Adapt the model to the support set, which has different classes."""
        # Dynamically adjust the final Dense layer to match the number of support classes
        self.model.pop()  # Remove the existing last layer
        self.model.add(Dense(num_support_classes, activation='softmax'))  # Add new output layer with num_support_classes units
        self.model.compile(optimizer=self.optimizer, loss=self.loss_function)


        # Fine-tune the model on the support set
        for step in range(self.num_adapt_steps):
            with tf.GradientTape() as tape:
                predictions = self.model(x_support, training=True)
                loss = self.loss_function(y_support, predictions)
            gradients = tape.gradient(loss, self.model.trainable_variables)
            gradient_norms = [tf.norm(grad).numpy() for grad in gradients if grad is not None]
            print(f"Adaptation Step {step + 1}, Gradient Norms: {gradient_norms}")
            self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))


    def predict_query(self, x_support, y_support, x_query,num_class):
        """Adapt to the support set and predict the labels for the query set."""
        self.adapt(x_support, y_support,num_class)
        predictions = self.model.predict(x_query, verbose=2)
        predicted_labels = np.argmax(predictions, axis=1)

        return predicted_labels

def predict_on_query_set(maml, support_data, support_labels, query_data, query_video_paths, output_csv_file,num_class):
    """Adapt model on support set and predict query data, saving results to CSV."""
    print("Adapting model to support set...")
    predicted_classes = maml.predict_query(support_data, support_labels, query_data,num_class)

    try:
        print(f"Saving predictions to {output_csv_file}...")
        with open(output_csv_file, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['VID_ID', 'LABEL'])
            for video_path, predicted_class in zip(query_video_paths, predicted_classes):
                writer.writerow([video_path, predicted_class])
    except Exception as e:
        print(f"Error saving predictions: {e}")



# Load your actual data here
support_data = features_test  # Load your support data
support_labels = labels_test    # Load your support labels
maml = MAML(LRCN_model, learning_rate=1e-4, meta_lr=1e-3, num_adapt_steps=4)
#name of the file to save csv result
output_csv_file = 'predictionsfinal.csv'
# Predict and save to CSV
predict_on_query_set(maml, support_data, support_labels, query_data, query_video_paths, output_csv_file,4)# 4 is the no. of the suppprt set class
